# Create Schema with a POST request

In [1]:
import weaviate

In [6]:
import json
import os

# Read the aquarium schema JSON file
schemas_path = os.path.join("..", "data", "cleaned-simple-3-collection-schemas.json")
with open(schemas_path, "r") as f:
    schemas = json.load(f)

In [7]:
import json
json_schema = json.loads(schemas[0])
json_schema["weaviate_collections"][0]

{'properties': [{'name': 'DishName',
   'data_type': ['string'],
   'description': 'The name of the dish offered in the restaurant menu.'},
  {'name': 'Price',
   'data_type': ['number'],
   'description': 'The price of the dish.'},
  {'name': 'IsVegetarian',
   'data_type': ['boolean'],
   'description': 'Indicates if the dish is vegetarian.'}],
 'class': 'RestaurantMenu',
 'description': 'The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.',
 'vectorIndexType': 'hnsw',
 'vectorizer': 'text2vec-transformers'}

In [8]:
json_schema

{'weaviate_collections': [{'properties': [{'name': 'DishName',
     'data_type': ['string'],
     'description': 'The name of the dish offered in the restaurant menu.'},
    {'name': 'Price',
     'data_type': ['number'],
     'description': 'The price of the dish.'},
    {'name': 'IsVegetarian',
     'data_type': ['boolean'],
     'description': 'Indicates if the dish is vegetarian.'}],
   'class': 'RestaurantMenu',
   'description': 'The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.',
   'vectorIndexType': 'hnsw',
   'vectorizer': 'text2vec-transformers'},
  {'properties': [{'name': 'CustomerName',
     'data_type': ['string'],
     'description': 'The name of the customer who places the order.'},
    {'name': 'TotalAmount',
     'data_type': ['number'],
     'description': "The total amo

In [9]:
import requests

url = "http://localhost:8080/v1/schema"

# Post each class schema separately
for class_schema in json_schema['weaviate_collections']:
    # Make sure the schema has all required fields
    if 'class' not in class_schema:
        print("Error: Schema missing required 'class' field")
        continue
        
    # Create a clean schema object with only the required fields
    clean_schema = {
        'class': class_schema['class'],  # Use existing 'class' field
        'description': class_schema.get('description', ''),
        'properties': [
            {
                'name': prop['name'],
                'description': prop.get('description', ''),
                'dataType': prop['data_type']  # Weaviate expects dataType, not data_type
            }
            for prop in class_schema.get('properties', [])
        ],
        'vectorizer': class_schema.get('vectorizer', 'text2vec-transformers'),
        'vectorIndexType': class_schema.get('vectorIndexType', 'hnsw'),
    }
    
    # Convert to string
    schema_str = json.dumps(clean_schema)
    
    print(f"Sending schema for class {clean_schema['class']}:")
    print(schema_str)
    
    response = requests.post(
        url, 
        data=schema_str,
        headers={'Content-Type': 'application/json'}
    )
    
    print(f"Response status: {response.status_code}")
    if response.status_code != 200:
        print(f"Error response: {response.text}")

Sending schema for class RestaurantMenu:
{"class": "RestaurantMenu", "description": "The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.", "properties": [{"name": "DishName", "description": "The name of the dish offered in the restaurant menu.", "dataType": ["string"]}, {"name": "Price", "description": "The price of the dish.", "dataType": ["number"]}, {"name": "IsVegetarian", "description": "Indicates if the dish is vegetarian.", "dataType": ["boolean"]}], "vectorizer": "text2vec-transformers", "vectorIndexType": "hnsw"}
Response status: 200
Sending schema for class CustomerOrders:
{"class": "CustomerOrders", "description": "The CustomerOrders collection captures details of customer orders, including their names, total amounts, and whether the orders are takeaway. This collection helps in pr

# Parse `collections.list_all()`

In [10]:
import weaviate

weaviate_client = weaviate.connect_to_local()

weaviate_client.collections.list_all()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 2.5.1.dev2434+g7589c2f. The latest version is 4.9.0.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


{'RestaurantMenu': _CollectionConfigSimple(name='RestaurantMenu', description='The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.', generative_config=None, properties=[_Property(name='dishName', description='The name of the dish offered in the restaurant menu.', data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WHITESPACE: 'whitespace'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=False), vectorizer='text2vec-transformers'), _Property(name='price', description='The price of the dish.', data_type=<DataType.NUMBER: 'number'>, index_filterable=True, index_range_filters=False, index_searchable=False, nested_properties=None, tokenization=None, vectorizer

In [11]:
def get_collections_info(client: weaviate.WeaviateClient) -> tuple[str, list[str]]:
    """
    Get detailed information about all collections in a Weaviate instance.
    
    Args:
        client: A Weaviate client instance
    
    Returns:
        tuple[str, list[str]]: Tuple containing formatted collection details string and list of collection names
    """
    
    collections = client.collections.list_all()
    
    # Get collection names as list
    collection_names = list(collections.keys())
    
    # Build output string
    output = []
    for collection_name, config in collections.items():
        output.append(f"\nCollection Name: {collection_name}")
        output.append(f"Description: {config.description}")
        output.append("\nProperties:")
        for prop in config.properties:
            output.append(f"- {prop.name}: {prop.description} (type: {prop.data_type.value})")
    
    return "\n".join(output), collection_names

# Call the function with our client and print result
info_str, collections_list = get_collections_info(weaviate_client)
print(info_str)


Collection Name: HospitalDepartments
Description: HospitalDepartments provides information on each department within the hospital, including their capacity and operational status. This helps in optimizing resource allocation and ensuring each department can handle patient load effectively.

Properties:
- departmentName: Name of the hospital department. (type: text)
- bedCapacity: Number of beds available in the department. (type: number)
- operational: Indicates if the department is currently operational. (type: boolean)

Collection Name: RestaurantMenu
Description: The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.

Properties:
- dishName: The name of the dish offered in the restaurant menu. (type: text)
- price: The price of the dish. (type: number)
- isVegetarian: Indicates if the dish i

# Create Function

OpenAI wants to take enum as an argument with the following:

`"enum": ["c", "f"]`

Function Schema:

```python
{
    "name": "search_weaviate_collection",
    "description": "Search a Weaviate Collection",
    "parameters": {
        "type": "object",
        "properties": {
            "collection_name": {
                "type": "enum": collections_list,
                "description": "The Weaviate Collection to search through."
            },
            "search_query": {
                "type": "string",
                "description": "The search query."
            }
        },
        "required": ["collection_name", "search_query"],
        "additionalProperties": False
    }
}
```

In [12]:
from typing import Literal, Optional, Dict, List, Union
from pydantic import BaseModel

class ParameterProperty(BaseModel):
    type: str
    description: str
    enum: Optional[List[str]] = None

class Parameters(BaseModel):
    type: Literal["object"]
    properties: Dict[str, ParameterProperty]
    required: Optional[List[str]]

class Function(BaseModel):
    name: str
    description: str
    parameters: Parameters

class Tool(BaseModel):
    type: Literal["function"]
    function: Function

# Example usage:
search_tool = Tool(
    type="function",
    function=Function(
        name="search_weaviate_collection",
        description="Search for the most relevant items to the provided `search_query` in a Weaviate Database Collection.",
        parameters=Parameters(
            type="object",
            properties={
                "collection_name": ParameterProperty(
                    type="string",
                    description="The Weaviate Collection to search through.",
                    enum=collections_list
                ),
                "search_query": ParameterProperty(
                    type="string",
                    description="The search query."
                )
            },
            required=["collection_name", "search_query"]
        )
    )
)

# Example of another tool:
calculate_avg_tool = Tool(
    type="function", 
    function=Function(
        name="calculate_average",
        description="Calculate the average of a numeric property across collection items",
        parameters=Parameters(
            type="object",
            properties={
                "collection_name": ParameterProperty(
                    type="string",
                    description="The Weaviate Collection to analyze",
                    enum=collections_list
                ),
                "property_name": ParameterProperty(
                    type="string",
                    description="The numeric property to average"
                )
            },
            required=["collection_name", "property_name"]
        )
    )
)

In [13]:
print(search_tool.model_dump_json(indent=2))

{
  "type": "function",
  "function": {
    "name": "search_weaviate_collection",
    "description": "Search for the most relevant items to the provided `search_query` in a Weaviate Database Collection.",
    "parameters": {
      "type": "object",
      "properties": {
        "collection_name": {
          "type": "string",
          "description": "The Weaviate Collection to search through.",
          "enum": [
            "HospitalDepartments",
            "RestaurantMenu",
            "MedicalStaff",
            "HospitalPatients",
            "CustomerOrders",
            "StaffMembers"
          ]
        },
        "search_query": {
          "type": "string",
          "description": "The search query.",
          "enum": null
        }
      },
      "required": [
        "collection_name",
        "search_query"
      ]
    }
  }
}


In [15]:
tools = [search_tool]

# Function Calling Test - Collection Query Routing

In [27]:
import openai

openai_client = openai.OpenAI(
    api_key = ""
)

prompt = "Does the restaurant have sandwhiches?"

messages = [
    {"role": "system", "content": "You are a helpful hospital support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": prompt}
]

response = openai_client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=messages,
    tools=tools,
)

In [28]:
print(response.choices[0].message.tool_calls[0].function.arguments)


{"collection_name":"RestaurantMenu","search_query":"sandwich"}
